In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text

from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from groupby_toolz import enrich_db, gcloud

import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
# from enrich_dimensions.params import parameters, query_from_file
# from enrich_dimensions.query_file import query_from_file 
# from enrich_dimensions.custom import custom_field 

# Read In Data

In [2]:
attributes=pd.read_csv('Data/90_affected_attributes.csv')
data=pd.read_csv('Data/90_syndicated_data.csv')
curated_data=pd.read_csv('Data/90_curated_data.csv')

# Print Out Attribute Names

In [3]:
print('Attributes')
print('####################')
attributes['attribute_name'].explode().value_counts()

Attributes
####################


Product Type              45
Material                  41
Color                     36
Features                  33
Count                     14
Capacity (oz)              9
Quantity                   6
Search Product Type        6
Power Type                 4
Tier Count                 4
Mount Type                 3
Package Quantity           3
Compartment Count          3
Wood Finish                2
Form                       2
Capacity (qt)              2
Back Style                 1
Board Type                 1
Ink Color                  1
Port Count                 1
Pencil Lead Number         1
Ply Count                  1
Design                     1
Rack Spacing               1
Capacity (gal)             1
Included Pockets           1
Point Style                1
Pencil Type                1
Size Width Cookware        1
Knife Type                 1
Care                       1
Pen Type                   1
Board Size                 1
Size Length Cookware       1
Tip Type      

# 1. Pause Buckets Associated With Attributes

In [4]:
att_buckets=attributes.groupby(['attribute_name', 'bucket_name']).sum()
att_buckets.to_csv('syn_att_bucket_pairs')
att_buckets[0:20]

Empty DataFrame
Columns: []
Index: [(Application Area, Cleaning Cloths), (Arm Style, High Chairs & Feeding, Booster, & Activity Seats), (Back Style, High Chairs & Feeding, Booster, & Activity Seats), (Board Size, Presentation Boards), (Board Type, Presentation Boards), (Capacity (gal), Fuel Storage Containers & Accessories), (Capacity (oz), Bowls), (Capacity (oz), Cocktail Preparation Tools), (Capacity (oz), Cruets & Sprayers), (Capacity (oz), Dredges/Shakers), (Capacity (oz), Food Storage Canisters/Containers & Accessories), (Capacity (oz), Measuring Cups), (Capacity (oz), Serving Utensils), (Capacity (oz), Specialty Serving Dishes), (Capacity (oz), Sugar Bowls/Jars/Dispensers), (Capacity (qt), Colanders & Strainers), (Capacity (qt), Measuring Cups), (Care, Glasses), (Cleaner Type, Cleaning Cloths), (Color, Aprons)]

# 2. Find Common Attributes Between Curated Data and Syndicated Data

In [5]:
# Get list of attributes
att_names=attributes['attribute_name'].explode().value_counts().reset_index() # List of attributes
mylist=att_names['index'].to_list()
att_list=list(map(lambda x: x.lower(), mylist)) #unique list
att_list.sort()

In [6]:
# Clean up the list of syndicated columns
syn_list=[*data][42:] 
syn_string = []
for i in syn_list:
    new_string=i.replace(r"__\w{0,4}_?_?", "")
#     new_string=i.replace(r"__\w{0,4}_?_?", "")
    syn_string.append(new_string)

In [7]:
# List of curated columns
merged=data.merge(curated_data.pivot(*curated_data).add_prefix('c:'), on='external_id')
merge_list=[*curated_data]

In [8]:
# Append curated columns to the Dataframe
cur_list=list(set(curated_data['snake_case_name']))
cur=[]
for i in range(len(cur_list)):
    cur.append('c:'+cur_list[i])

for i in range(len(cur)):
    syn_string.append(cur[i])

In [9]:
# Compare the syndicated columns with the attributes
syn_col=[*data][42:]
print('###########################')
for i in range(len(att_list)):
    x=att_list[i][0:5]
    print(att_list[i].upper())
    print('--------------')
    for i in range(len(syn_string)):
        z=re.match(fr"(?i).*{x}.*",syn_string[i])
        if z:
            print((z.group()))
    print('###########################')

###########################
APPLICATION AREA
--------------
###########################
ARM STYLE
--------------
###########################
BACK STYLE
--------------
###########################
BOARD SIZE
--------------
Board Height [Nom]
Board Depth [Nom]
Board Type
Board Size (W x H)
Board Width [Nom]
Board Eraser Type
###########################
BOARD TYPE
--------------
Board Height [Nom]
Board Depth [Nom]
Board Type
Board Size (W x H)
Board Width [Nom]
Board Eraser Type
###########################
CAPACITY (GAL)
--------------
s:capacity
Capacity (Volume) [Nom]
Shelf Capacity (Weight) [Nom]
c:weight_capacity
c:shelf_weight_capacity
###########################
CAPACITY (OZ)
--------------
s:capacity
Capacity (Volume) [Nom]
Shelf Capacity (Weight) [Nom]
c:weight_capacity
c:shelf_weight_capacity
###########################
CAPACITY (QT)
--------------
s:capacity
Capacity (Volume) [Nom]
Shelf Capacity (Weight) [Nom]
c:weight_capacity
c:shelf_weight_capacity
##########################

# Compare Curated Data With Syndicated Data

### Print out list of rows that had different values

### Board Type--Not good comparison

In [10]:
# merged['junk']=merged[[
# 'Board Height [Nom]',
# 'Board Depth [Nom]',
# 'Board Type',
# 'Board Size (W x H)',
# 'Board Width [Nom]',
# 'Board Eraser Type'
# ]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# merged['junk'].explode().value_counts()

### Capacity--Gallons, Ounces, Quarts

In [11]:
# 32 ounces in a quart
merged['junk']=merged[[
's:capacity',
'Capacity (Volume) [Nom]',
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

                                90
8 ounces_________8 oz.           4
24 oz._________24 Ounces         4
5 ounces_________5 oz.           4
24 ounces_________32 oz.         4
2.5 Liters                       4
8 Quarts                         3
9-1/2 ounces_________9.5 oz.     3
8 Ounces                         3
13 Ounces                        2
3 Ounces                         2
41 Ounces                        2
10 Ounces                        2
15 Ounces                        2
6 Ounces                         2
22 Ounces                        2
52 Ounces                        2
10 Pounds                        2
12 oz._________12 ounces         2
20 oz._________20 ounces         2
16 ounces_________16 oz.         2
16 Ounces_________16 oz.         2
16 ounces                        2
11 Ounces                        2
800 Pounds                       2
64 Ounces                        2
32 Ounces                        2
7 Ounces                         2
12 Ounces           

### Capacity---Weight

In [12]:
merged['junk']=merged[[
'Shelf Capacity (Weight) [Nom]',
'c:weight_capacity',
'c:shelf_weight_capacity'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

                           179
800 lbs._________800 lb      2
Name: junk, dtype: int64

### Color

In [13]:
merged['junk']=merged[[
's:color',
's:handle_color',
'Surface Color',
'Color',
'Frame Color',
'Upholstery Color',
's:colors',
'c:color'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

Clear                                                                    42
White                                                                    26
Silver                                                                   26
Tan                                                                      16
Black                                                                    14
Red                                                                      10
Blue                                                                     10
Amber                                                                     8
Silver_________["Black","Silver"]                                         3
Mahogany_________Brown                                                    3
Tan_________Natural                                                       2
Brown_________Walnut                                                      2
Mahogany_________Black/Mahogany_________Black_________["Black","Red"]     2
Brown       

### Compartments

In [14]:
merged['junk']=merged[[
's:number_of_compartments',
# 's:compatibility',
's:compartments',
'c:compartment_count'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

       170
4.0      5
6.0      5
5.0      1
Name: junk, dtype: int64

### Count--Not good comparison

In [15]:
# merged['junk']=merged[[
# # 's:country_of_origin',
# 'c:count',
# 'c:shelf_count',
# 'c:ply_count',
# # 'c:tier_count',
# 'c:sheet_count',
# 'c:compartment_count'
# ]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# merged['junk'].explode().value_counts()

### Eraser Type

In [16]:
merged['junk']=merged[[
'Board Eraser Type',
'c:eraser_type'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

             180
Dry Erase      1
Name: junk, dtype: int64

### Features

In [17]:
merged['junk']=merged[[
's:features',
'c:features'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

                                                                                                                                                 103
[NSF Listed, Dishwasher Safe, Heat Resistant to 212?? F]                                                                                          31
[Tumbler, Wide Rim, Stackable, Dishwasher Safe]                                                                                                   26
[Cup and Lid Organizer]                                                                                                                            4
Includes Lid                                                                                                                                       3
[Fluted Finish, Swirl Pattern, BPA Free, Dishwasher Safe]                                                                                          2
["Lockable Wheels","Rolling","Built-In Casters","Built-In Brake"]                                         

### Material

In [18]:
merged['junk']=merged[[
's:base_material',
# 's:hazardous_material',
's:handle_material',
's:material',
's:collar_material',
's:liner_material',
'Surface Material',
'Frame Material',
'Material(s)',
's:cap_material',
's:top_material',
's:stopper_material',
's:pourer_material',
's:bottom_material',
'c:material'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

Plastic                                              46
Melamine                                             33
Polycarbonate                                        24
Stainless Steel                                      15
Glass_________Stainless Steel                         9
Wood_________Rubberwood                               8
Wood_________Melamine                                 5
Card Stock                                            4
ABS Plastic                                           4
Plastic_________SAN Plastic                           3
Cotton/Poly Blend                                     3
Glass                                                 3
Plastic_________Stainless Steel                       2
Cotton                                                2
Aluminum                                              2
Stainless Steel_________Polycarbonate                 2
Glass_________Chrome                                  2
Chrome Plated_________Metal_________Chrome      

### Mount Type

In [19]:
merged['junk']=merged[[
's:mount_style',
'c:mount_type'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

                177
Freestanding      4
Name: junk, dtype: int64

### Pen Type

In [20]:
merged['junk']=merged[[
'c:pencil_lead_number',
'c:pencil_type'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

    181
Name: junk, dtype: int64

### Product Type

In [21]:
merged['junk']=merged[[
'c:search_product_type',
'c:product_type'
]].apply(lambda x: "_________".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
merged['junk'].explode().value_counts()

                                          142
Decanters                                   6
High Chairs                                 5
Project Boards                              4
Cup Organizers                              4
Tickets                                     4
Booster Seats                               3
Call Bells                                  2
Kitchen Towels                              2
["Cart","Carts"]_________Utility Carts      2
Clothes Hangers                             1
Shop Towels                                 1
Chalkboards                                 1
Markers                                     1
Bar Mats & Liners                           1
Utility Carts                               1
Pitchers                                    1
Name: junk, dtype: int64

# Cleaning up the Columns

In [22]:
#######################################################################    Normalizing Color    ##########################################################################
colorSplitRegex = r'-| |/|\\|,'
# G E T   A   C O L O R   N A M E
# get the closest color from an rgb tuple
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]
# get the color name either from webcolors or closest_color
def get_colour_name(requested_colour):
    try:
        closest_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
    return closest_name.title()
# G E T   A   C O L O R   V A L U E
# convert a string to an rgb tuple
def parseColor(string):
    try:
        # convert the color string to rgb, default spec is css3
        return webcolors.name_to_rgb(string)
    except Exception:
        pass
# parse raw data for strings we can match on
# if the string has a dash, slash, or space try each color individually
# and average out the RGB values with a simple a+b+c/n forumla
def parseColorData(data):
    # init an empty array for color values
    colorValues = []
    # split it on dashes, spaces, slashes, or commas
    if re.split(colorSplitRegex, data):
        for color in re.split(colorSplitRegex, data):
            colorValues.append(parseColor(color))
    # if it can't be split just full send it
    else:
        colorValues.append(parseColor(color))
    # remove None values
    colorValues = [
        colorValue for colorValue in colorValues if colorValue is not None]
    if not len(colorValues):
        return data
    # find the average color and convert it to a color name
    #######################################################
    # init an average color value
    averageColor = [0, 0, 0]
    for colorValue in colorValues:
        for index, value in enumerate(colorValue):
            averageColor[index] += value
    averageColor = [math.floor(value / (len(colorValues) or 1))
                    for value in averageColor]
    # just change this to a return to run this function
    return get_colour_name(averageColor)




############################################################################ Convert Fractions to decimal###############################################################
from decimal import *
TWOPLACES = Decimal(10) ** -2
from fractions import Fraction
def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()
def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]




############################################################################ Convert Fractions in form of a date ###############################################################
# Excel might convert 8-1/2 to 08/01/2002 and we need to conver this back into 8.5  
def date_to_decimal(numbers):
    if numbers.find('/') > -1:
        try:
            numbers=str(numbers)
            one, two,three = numbers.split('/')
            tmp = float(two)/float(three[-1])
            num = int(one) + tmp
            return str(num)
        except:
            return numbers
    else:
        return numbers

# Convert a fraction "1/4" that is written as a date "1-4-2021"
def remove_basic_date(numbers):
    if numbers.find('-2021') > -1:
        first, second, year = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
    

# Convert a fraction "1/4" that is written as a date "2021-01-04"
def remove_date_year_first(numbers):
    if numbers.find('2021-') > -1:
        year, first, second = numbers.split('-')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
    
# Convert a fraction "1/4" that is written as a date "1/4/2021"    
def remove_date_year_last(numbers):
    if numbers.find('/2021') > -1:
        first, second,year = numbers.split('/')
        num=float(first)/float(second)
        return str(num)
    else:
        return numbers
regex_year_month_day = r"(?i)(\d\d\d\d\s\d\d\s\d\d)|()"     




# TEMP RANGES
#########################################################################
# Converts Above 40 Degrees F  and Above 0
def above_freezing(numbers): 
    if numbers.find('above freezing') > -1:   
        numbers=32
        return numbers
    elif numbers.find('above ') > -1:   
        numbers=re.sub(r'[A-z]|®|™|Ã|¯|Â|¿|Å|“|\"|�|•| ','',str(numbers))
        return numbers
    elif numbers.find('Greater ') > -1:   
        numbers=re.sub(r'[A-z]|®|™|Ã|¯|Â|¿|Å|“|\"|�|•| ','',str(numbers))
        return numbers
    else:
        return numbers
# data['min']=data['min'].apply(lambda x: above_freezing(x))

# Splits temp ranges: 60 to 100 Degrees F, -30 to 200 Degrees F, 50-100 Degrees
def split_height(data, col, dimension):
    data[col]=data[col].apply(lambda x:str(x).lower()) 
    data[['min_'+dimension, 'max_'+dimension]]=data[col].str.split('to',expand=True,)
    data['min_'+dimension]=data['min_'+dimension].apply(lambda x: above_freezing(x)).apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•','',str(measure)))
    data['max_'+dimension]=data['max_'+dimension].apply(lambda measure: re.sub(r'[a-z]|[A-z]|®|™|Ã|¯|Â|¿|Å|“|\"|�|•| |∩┐╜|\+|°','',str(measure)))   
    
def split_temp(data, col, dimension):
    data[col]=data[col].apply(lambda x:x.lower())
    data[['min_'+dimension, 'max_'+dimension]]=data[col].str.split(' to |(?<=\d)\s?-|max|below|less(?=u)',expand=True,)
    data['min_'+dimension]=data['min_'+dimension].apply(lambda x: above_freezing(x)).apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•|°| ','',str(measure)))
    data['max_'+dimension]=data['max_'+dimension].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\"|�|•| |∩┐╜|\+|°','',str(measure)))
    
def add_unit(df, col, unit):
    df[col] = df[col].fillna('').astype(str) + unit
    df.loc[df[col] == unit, col] = ''
    
def remove_metric(numbers, inputs):
    if str(numbers).find(inputs) > -1:
        numbers=''
        return numbers
    else:
        return numbers
    
    
    
######### REPLACE TEXT #########
# Replace specific text in a string: Exmapel1  to  Example1
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text
# replacement={'Exmapel1':'Example1',    }
# df= df.apply(lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# or you can just do .replace('Exmapel1','Example1')
# or you can do .apply(lambda x: re.sub(r'''Exmapel1''','Example1',str(x)))





##### Extract #######
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))




######## Fix Encoding Problems #########
import unicodedata
def convert_ascii(x):
    return str(unicodedata.normalize('NFKC', x).encode('ascii','ignore')).replace('\\','')

######## Import Curation Code ##############
from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, clean_data,reg_ex 




################## Remove Duplicates ##################
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
# power[original_col]=power[original_col].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))

# Merging two Columns

In [23]:
# # make a unique copy of the dataframe
# import copy
# power = copy.copy(merged)

# # Assign column names
# attribute='Capacity (Volume) [Nom]'
# original_col='s:capacity'
# col = attribute

# ################## CURATE COLUMN 1 ##################
# # power[col]=power[col].apply(lambda x: date_to_decimal(x))#.apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x))
# power[col]=power[col].apply(lambda x: re.sub(r'#|\+|\-','',str(x))).apply(lambda x: re.sub(r'(?i)"|In',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'(?i)lb\.|LB|lbs\.?|pounds?','lb',str(x))).apply(lambda x: re.sub(r'(?i)oz\.|ounces?','oz',str(x))).apply(lambda x: re.sub(r'N-m','Nm',str(x))).apply(lambda x: re.sub(r'(?i)Quarts?','qt',str(x))).apply(lambda x: re.sub(r'(?i)liters?','liters',str(x))).apply(lambda x: re.sub(r'\(1 qt\)','',str(x))).apply(lambda x: re.sub(r'(?<=\w)\-(?=\w)',' ',str(x)))                                                                                       
# power[col]=power[col].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
# power[col]=power[col].apply(lambda x: re.sub(r'(?<=[A-z])s?\.','',str(x)))
# power[col]=power[col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
# power[col]=power[col].apply(lambda x: convert_ascii(x)).apply(lambda x: re.sub(r"b'",'',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# power[col]=power[col].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# rounds(power,col)
# power[col]=power[col].apply(lambda x: re.sub(r"\[|\]|'",'',str(x)))



# ################## CURATE COLUMN 2 ##################
# # power[original_col]=power[original_col].apply(lambda x: date_to_decimal(x))#.apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x))
# power[original_col]=power[original_col].apply(lambda x: re.sub(r'#|\+|\-','',str(x))).apply(lambda x: re.sub(r'(?i)"|In',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'(?i)lb\.|LB|lbs\.?|pounds?','lb',str(x))).apply(lambda x: re.sub(r'(?i)oz\.|ounces?','oz',str(x))).apply(lambda x: re.sub(r'N-m','Nm',str(x))).apply(lambda x: re.sub(r'(?i)Quarts?','qt',str(x))).apply(lambda x: re.sub(r'(?i)liters?','liters',str(x))).apply(lambda x: re.sub(r'\(1 qt\)','',str(x))).apply(lambda x: re.sub(r'(?<=\w)\-(?=\w)',' ',str(x)))                                                                                       
# power[original_col]=power[original_col].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
# power[original_col]=power[original_col].apply(lambda x: re.sub(r'(?<=[A-z])s?\.','',str(x)))
# power[original_col]=power[original_col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
# power[original_col]=power[original_col].apply(lambda x: convert_ascii(x)).apply(lambda x: re.sub(r"b'",'',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))

# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# power[original_col]=power[original_col].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# rounds(power,original_col)
# power[original_col]=power[original_col].apply(lambda x: re.sub(r"\[|\]|'",'',str(x)))


# ################## Combining The Columns ##################

# # Assign a final column of the combined two other columns
# final=f'Q:{col}'
# power[final]=power[[col, original_col]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
# power[final]=power[final].apply(lambda x: re.sub(r'nan__|__nan','',str(x)))

# # Sort out the different columns
# pat = r"__"               
# power['difference'] = power[final].apply(lambda x: re_extract(pat, x))
# power=power[power['difference'].astype(str)=='[]']

In [24]:
# make a unique copy of the dataframe
import copy
power = copy.copy(merged)

# Assign column names
attribute='Capacity (Volume) [Nom]'
original_col='s:capacity'
col = attribute

def curate_col(power,col):
    ################## CURATE COLUMN 1 ##################
    # power[col]=power[col].apply(lambda x: date_to_decimal(x))#.apply(lambda x: remove_fraction(x)).apply(lambda x: remove_frac(x)).apply(lambda x: re_extract(regex_year_month_day,x))
    power[col]=power[col].apply(lambda x: re.sub(r'#|\+|\-','',str(x))).apply(lambda x: re.sub(r'(?i)"|In',' in',str(x))).apply(lambda x: re.sub(r'(?i)g\/l',' g/l',str(x))).apply(lambda x: re.sub(r'(?i)lb\.|LB|lbs\.?|pounds?','lb',str(x))).apply(lambda x: re.sub(r'(?i)oz\.|ounces?','oz',str(x))).apply(lambda x: re.sub(r'N-m','Nm',str(x))).apply(lambda x: re.sub(r'(?i)Quarts?','qt',str(x))).apply(lambda x: re.sub(r'(?i)liters?','liters',str(x))).apply(lambda x: re.sub(r'\(1 qt\)','',str(x))).apply(lambda x: re.sub(r'(?<=\w)\-(?=\w)',' ',str(x)))                                                                                       
    power[col]=power[col].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
    power[col]=power[col].apply(lambda x: re.sub(r'(?<=[A-z])s?\.(?!\w)','',str(x)))
    power[col]=power[col].str[2:-2].str.split('","').apply(lambda x: '["'+'","'.join(natsorted(x))+'"]')
    power[col]=power[col].apply(lambda x: convert_ascii(x)).apply(lambda x: re.sub(r"b'",'',str(x))).apply(lambda x: re.sub(r"\]'",']',str(x)))
    import ast
    def remove_duplicates(A):
        [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
        return A
    power[col]=power[col].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
    rounds(power,col)
    power[col]=power[col].apply(lambda x: re.sub(r"\[|\]|'",'',str(x)))

curate_col(power,col)
curate_col(power,original_col)

################## Combining The Columns ##################

# Assign a final column of the combined two other columns
final=f'Q:{col}'
power[final]=power[[col, original_col]].apply(lambda x: "__".join(set([str(i) for i in x if pd.notnull(i)])), axis=1)
power[final]=power[final].apply(lambda x: re.sub(r'nan__|__nan','',str(x)))

# Sort out the different columns
pat = r"__"               
power['difference'] = power[final].apply(lambda x: re_extract(pat, x))
power=power[power['difference'].astype(str)=='[]']

In [25]:
power[final].explode().value_counts()

nan           90
8 oz           7
16 oz          6
12 oz          6
5 oz           5
2.5 liters     4
24 oz          4
8 qt           3
20 oz          3
64 oz          3
32 oz          3
3 oz           2
800 lb         2
41 oz          2
10 lb          2
0.5 oz         2
15 oz          2
3 liters       2
11 oz          2
10 oz          2
22 oz          2
13 oz          2
6 oz           2
7 oz           2
2 oz           2
52 oz          2
1.6 qt         1
0.53 qt        1
4 qt           1
5 lb           1
45.5 oz        1
1 oz           1
96 oz          1
2 qt           1
9 oz           1
Name: Q:Capacity (Volume) [Nom], dtype: int64